## Parafac

Now the CANDECOMP/PARAFAC decomposition. The above tensor is too high a rank to reasonably decompose in this example, so we instead generate an example sparse tensor from a random sparse factorization and re-factor it.

In [1]:
import numpy as np
import sparse
import tensorly as tl

In [2]:
shape = (1000, 1001, 1002, 100)
rank = 5

starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors

[<COO: shape=(1000, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1001, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1002, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(100, 5), dtype=float64, nnz=5, fill_value=0.0>]

Now convert it to a tensor. It is very important to use `kruskal_to_tensor` from the sparse backend, as a fully dense version of the tensor would use several TB of memory.

In [3]:
from tensorly.contrib.sparse.kruskal_tensor import kruskal_to_tensor
tensor = kruskal_to_tensor(starting_factors)
tensor

<COO: shape=(1000, 1001, 1002, 100), dtype=float64, nnz=6228, fill_value=0.0>

As before, we can compare the actual spase used by the tensor vs. what it would require if it were dense.

In [4]:
tensor.nbytes / 1e9                # Actual memory usage in GB

0.00024912

In [5]:
np.prod(tensor.shape) * 8 / 1e9    # Memory usage if array was dense, in GB

802.4016

In [6]:
import time
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [7]:
from tensorly.decomposition import parafac

Now we can decompose the tensor. Note again how much memory is actually used.

In [8]:
%%memit
start_time = time.time()
factors = parafac(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

Starting iteration 0
Mode 0 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 1 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 2 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 3 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
reconstruction error=0.3807770168041797
Starting iteration 1
Mode 0 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 1 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 2 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 3 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
reconstruction error=0.0816367702253021, variation=0.2991402465788776.
Starting iteration 2
Mode 0 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 1 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 2 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank

In [9]:
type(factors[0])

numpy.ndarray

In [10]:
[i.shape for i in factors]

[(1000, 5), (1001, 5), (1002, 5), (100, 5)]

Note that even though we started with a sparse tensor, the factors are dense. This is because we used the dense version of `parafac`. Since the factors are in general dense, even for a sparse tensor, this is generally preferred. 

We can also use the dense version of `parafac`. It should give the same answer, though it may be slower.

In [11]:
from tensorly.contrib.sparse.decomposition import parafac as parafac_sparse

In [12]:
%%memit
start_time = time.time()
factors_sparse = parafac_sparse(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

Starting iteration 0
Mode 0 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 1 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 2 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 3 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
reconstruction error=0.4707823027303014
Starting iteration 1
Mode 0 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 1 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 2 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 3 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
reconstruction error=0.05795342675835725, variation=0.4128288759719442.
Starting iteration 2
Mode 0 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 1 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Rank 3 of 5
 Rank 4 of 5
Mode 2 of 4
 Rank 0 of 5
 Rank 1 of 5
 Rank 2 of 5
 Ran

Let's look at the result

In [13]:
factors_sparse

[<COO: shape=(1000, 5), dtype=float64, nnz=110, fill_value=0.0>,
 <COO: shape=(1001, 5), dtype=float64, nnz=110, fill_value=0.0>,
 <COO: shape=(1002, 5), dtype=float64, nnz=135, fill_value=0.0>,
 <COO: shape=(100, 5), dtype=float64, nnz=20, fill_value=0.0>]

Because the `factors_sparse` are sparse, we can reconstruct them into a tensor without using too much memory. In general, this will not be the case, but it is for our toy example. Let's do this to look at the absolute error for the decomposition. 

In [14]:
tl.norm(tensor - kruskal_to_tensor(factors_sparse))

0.6266681146964519

It is not actually necessary to compute this, as the same as the norm of the tensor times the reconstruction error that was printed by the algorithm (you can pass `return_errors=True` to `parafac()` to have the reconstruction errors be returned along with the factors). That is, $$\mathrm{reconstruction\ error} = \frac{\|\mathrm{tensor} - \mathrm{kruskal\_to\_tensor}(\mathrm{factors})\|_2}{\|\mathrm{tensor}\|_2}$$ (they won't be exactly the same due to numerical differences in how they are calculated).

In [15]:
tl.norm(tensor - kruskal_to_tensor(factors_sparse))/tl.norm(tensor)

0.05634239215946769

Let's look at one of the nonzero entries to see how close it is to the original tensor. The factors satisfy $$\sum_{r=0}^{R-1} {f_0}_r\circ {f_1}_r \circ {f_2}_r \circ {f_3}_r,$$ where $R$ is the rank (here 5), ${f_i}_r$ is the $r$-th column of the $i$-th factor of the decomposition, and $\circ$ is the vector outer product. Component-wise, this translates to a product of corresponding elements per component for each factor, summed over the columns.

In [16]:
tensor.coords

array([[  8,   8,   8, ..., 949, 949, 949],
       [ 10,  10,  10, ..., 846, 846, 846],
       [  4,  48,  62, ..., 989, 989, 989],
       [ 69,  69,  69, ...,  52,  66,  81]])

In [17]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.0749382519583567

In [18]:
dense_val = np.sum(np.prod(np.stack([factors[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
dense_val

0.07493825195799829

And the same for the sparse factors

In [19]:
sparse_val = np.sum(np.prod(sparse.stack([factors_sparse[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
sparse_val

0.0749382519514725

In [20]:
np.abs(orig_val - dense_val)

3.5840774792461616e-13

In [21]:
np.abs(orig_val - sparse_val)

6.884201542156632e-12

The difference here is mostly due to random chance. The total reconstruction errors for the two runs of algorithm were roughly the same. In general, the error of the factorization will vary due to the randomness of the initial factors chosen by the algorithm.